# Assembly: functions
_COSC 208, Introduction to Computer Systems, 2023-03-29_

## Announcements
* Project 3 due tomorrow @ 11pm
* Exam 2
    * Study guide posted on Moodle
    * Take-home portion: released after class on Friday; due at the beginning of class on Monday
    * In-class portion: during class on Monday

## Outline
* Warm-up
* Functions

## Warm-up

Q1: _The following C code was compiled into assembly (using `clang`)._ 

In [ ]:
/*1*/   int adjust(int value) {
/*2*/       if (value < 10) {
/*3*/           value = value * 10;
/*4*/       }
/*5*/       else {
/*6*/           value = value / 10;
/*7*/       }
/*8*/       return value;
/*9*/   }

_For each line of assembly:_
1. _Indicate which original line of C code (above) the assembly instruction was derived from; and_
2. _Write the low-level C code equivalent of the assembly instruction, treating registers as if they were variable names._

```
0000000000400544 <adjust>:                          // Line     Low-level C
    400544:  d10043ff   sub  sp, sp, #0x10          // 1        sp = sp - 0x10
    400548:  b9000fe0   str  w0, [sp, #12]          // 1        *(sp+12) = w0
    40054c:  b9400fe8   ldr  w8, [sp, #12]          // 2        w8 = *(sp+12)
    400550:  7100291f   cmp  w8, #0xa               // 2        if w8 >= 10
    400554:  540000ca   b.ge 40056c <adjust+0x28>   // 2            goto 40065c
    400558:  b9400fe8   ldr  w8, [sp, #12]          // 3        w8 = *(sp+12)
    40055c:  52800149   mov  w9, #0xa               // 3        w9 = 10
    400560:  1b097d08   mul  w8, w8, w9             // 3        w8 = w8 * w9
    400564:  b9000fe8   str  w8, [sp, #12]          // 3        *(sp+12) = w8
    400568:  14000005   b    40057c <adjust+0x38>   // 4        goto 40057c
    40056c:  b9400fe8   ldr  w8, [sp, #12]          // 6        w8 = *(sp+12)
    400570:  52800149   mov  w9, #0xa               // 6        w9 = 10
    400574:  1ac90d08   sdiv w8, w8, w9             // 6        w8 = w8 / w9
    400578:  b9000fe8   str  w8, [sp, #12]          // 6        *(sp+12) = w8
    40057c:  b9400fe0   ldr  w0, [sp, #12]          // 8        w0 = *(sp+12)
    400580:  910043ff   add  sp, sp, #0x10          // 8        sp = sp + 0x10
    400584:  d65f03c0   ret                         // 8        return
```

```
0000000000400544 <adjust>:                          // Line     Low-level C
    400544:  d10043ff   sub  sp, sp, #0x10          //
    400548:  b9000fe0   str  w0, [sp, #12]          //
    40054c:  b9400fe8   ldr  w8, [sp, #12]          //
    400550:  7100291f   cmp  w8, #0xa               //
    400554:  540000ca   b.ge 40056c <adjust+0x28>   //
    400558:  b9400fe8   ldr  w8, [sp, #12]          //
    40055c:  52800149   mov  w9, #0xa               //
    400560:  1b097d08   mul  w8, w8, w9             //
    400564:  b9000fe8   str  w8, [sp, #12]          //
    400568:  14000005   b    40057c <adjust+0x38>   //
    40056c:  b9400fe8   ldr  w8, [sp, #12]          //
    400570:  52800149   mov  w9, #0xa               //
    400574:  1ac90d08   sdiv w8, w8, w9             //
    400578:  b9000fe8   str  w8, [sp, #12]          //
    40057c:  b9400fe0   ldr  w0, [sp, #12]          //
    400580:  910043ff   add  sp, sp, #0x10          //
    400584:  d65f03c0   ret                         //
```

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Function calls in assembly

* Calling conventions
    * _In which registers are parameters stored?_ — `x0`/`w0`, `x1`/`w1`, `x2`/`w2`, ...
    * _In which register is the return value stored?_ — `x0`/`w0`
    * Return address stored in `x30`
    * Caller's return address stored in callee's stack frame
* Use `bl` (branch & link) to make a function call
    * Store `pc+4` in `x30`
    * Update `pc` to specified code address
* Example

In [ ]:
/*1*/  int multiply(int a, int b) {
/*2*/      int c = a * b;
/*3*/      return c;
/*4*/  }
/*5*/  int volume(int x, int y, int z) {
/*6*/      int w = multiply(x, y);
/*7*/      w = multiply(w, z);
/*8*/      return w;
/*9*/  }

```
0000000000400544 <multiply>:                        // Line     Low-level C
    400544:    d10043ff     sub    sp, sp, #0x10    // 1        sp = sp - 0x10
    400548:    b9000fe0     str    w0, [sp, #12]    // 1        *(sp+12) = w0
    40054c:    b9000be1     str    w1, [sp, #8]     // 1        *(sp+12) = w1
    400550:    b9400fe8     ldr    w8, [sp, #12]    // 2        w8 = *(sp+12)
    400554:    b9400be9     ldr    w9, [sp, #8]     // 2        w9 = *(sp+8)
    400558:    1b097d08     mul    w8, w8, w9       // 2        w8 = w8 * w9
    40055c:    b90007e8     str    w8, [sp, #4]     // 2        *(sp+4) = w8
    400560:    b94007e0     ldr    w0, [sp, #4]     // 3        w0 = *(sp+4)
    400564:    910043ff     add    sp, sp, #0x10    // 3        sp = sp + 0x10
    400568:    d65f03c0     ret                     // 3        pc = x30
```

```
000000000040056c <volume>:                          // Line     Low-level C
    40056c:    d10083ff     sub    sp, sp, #0x20    // 5        sp = sp - 0x20
    400570:    f9000bfe     str    x30, [sp, #16]   // 5        *(sp+16) = x30
    400574:    b9000fe0     str    w0, [sp, #12]    // 5        *(sp+12) = w0
    400578:    b9000be1     str    w1, [sp, #8]     // 5        *(sp+8) = w1
    40057c:    b90007e2     str    w2, [sp, #4]     // 5        *(sp+4) = w2
    400580:    b9400fe0     ldr    w0, [sp, #12]    // 6        w0 = *(sp+12)
    400584:    b9400be1     ldr    w1, [sp, #8]     // 6        w1 = *(sp+8)
    400588:    97ffffef     bl    400544 <multiply> // 6        x30 = pc + 4; pc = 0x400544
    40058c:    b90003e0     str    w0, [sp]         // 6        *sp = w0
    400590:    b94003e0     ldr    w0, [sp]         // 7        w0 = *sp
    400594:    b94007e1     ldr    w1, [sp, #4]     // 7        w1 = *(sp+4)
    400598:    97ffffeb     bl    400544 <multiply> // 7        x30 = pc + 4; pc = 0x400544
    40059c:    b90003e0     str    w0, [sp]         // 7        *sp = w0
    4005a0:    b94003e0     ldr    w0, [sp]         // 8        w0 = *sp
    4005a4:    f9400bfe     ldr    x30, [sp, #16]   // 8        x30 = *(sp+16)
    4005a8:    910083ff     add    sp, sp, #0x20    // 8        sp = sp + 0x20
    4005ac:    d65f03c0     ret                     // 8        pc = x30
```

```
0000000000400544 <multiply>:                           // Line     Low-level C
    400544:    d10043ff     sub    sp, sp, #0x10       //
    400548:    b9000fe0     str    w0, [sp, #12]       //
    40054c:    b9000be1     str    w1, [sp, #8]        //
    400550:    b9400fe8     ldr    w8, [sp, #12]       //
    400554:    b9400be9     ldr    w9, [sp, #8]        //
    400558:    1b097d08     mul    w8, w8, w9          //
    40055c:    b90007e8     str    w8, [sp, #4]        //
    400560:    b94007e0     ldr    w0, [sp, #4]        //
    400564:    910043ff     add    sp, sp, #0x10       //
    400568:    d65f03c0     ret                        //
```

```
000000000040056c <volume>:                             // Line     Low-level C
    40056c:    d10083ff     sub    sp, sp, #0x20       //
    400570:    f9000bfe     str    x30, [sp, #16]      //
    400574:    b9000fe0     str    w0, [sp, #12]       //
    400578:    b9000be1     str    w1, [sp, #8]        //
    40057c:    b90007e2     str    w2, [sp, #4]        //
    400580:    b9400fe0     ldr    w0, [sp, #12]       //
    400584:    b9400be1     ldr    w1, [sp, #8]        //
    400588:    97ffffef     bl     400544 <multiply>   // 
    40058c:    b90003e0     str    w0, [sp]            //
    400590:    b94003e0     ldr    w0, [sp]            //
    400594:    b94007e1     ldr    w1, [sp, #4]        //
    400598:    97ffffeb     bl     400544 <multiply>   // 
    40059c:    b90003e0     str    w0, [sp]            //
    4005a0:    b94003e0     ldr    w0, [sp]            //
    4005a4:    f9400bfe     ldr    x30, [sp, #16]      //
    4005a8:    910083ff     add    sp, sp, #0x20       //
    4005ac:    d65f03c0     ret                        //
```

<div style="page-break-after:always;"></div>

Q2: _The following C code was compiled into assembly (using `clang`)._

In [ ]:
/*10*/  int main() {
/*11*/      int n = volume(1, 3, 5);
/*12*/      return n; 
/*13*/  }

_For each line of assembly:_
1. _Indicate which original line of C code (above) the assembly instruction was derived from; and_
2. _Write the low-level C code equivalent of the assembly instruction, treating registers as if they were variable names._

```
00000000004005b0 <main>:                            // Line     Low-level C
    4005b0:    d10083ff     sub    sp, sp, #0x20    // 10       sp = sp - 0x20
    4005b4:    f9000bfe     str    x30, [sp, #16]   // 10       *(sp+16) = x30
    4005b8:    52800020     mov    w0, #0x1         // 11       w0 = 0x1
    4005bc:    52800061     mov    w1, #0x3         // 11       w1 = 0x3
    4005c0:    528000a2     mov    w2, #0x5         // 11       w2 = 0x5
    4005c4:    97ffffe9     bl    40056c <volume>   // 11       x30 = pc + 4; pc = 0x40056c
    4005c8:    b9000be0     str    w0, [sp, #8]     // 11       *(sp+8) = w0
    4005cc:    b9400be0     ldr    w0, [sp, #8]     // 12       w0 = *(sp+8)
    4005d0:    f9400bfe     ldr    x30, [sp, #16]   // 12       x30 = *(sp+16)
    4005d4:    910083ff     add    sp, sp, #0x20    // 12       sp = sp + 0x20
    4005d8:    d65f03c0     ret                     // 12       pc = x30
```

```
00000000004005b0 <main>:                               // Line     Low-level C
    4005b0:    d10083ff     sub    sp, sp, #0x20       //
    4005b4:    f9000bfe     str    x30, [sp, #16]      //
    4005b8:    52800020     mov    w0, #0x1            //
    4005bc:    52800061     mov    w1, #0x3            //
    4005c0:    528000a2     mov    w2, #0x5            //
    4005c4:    97ffffe9     bl     40056c <volume>     //
    4005c8:    b9000be0     str    w0, [sp, #8]        //
    4005cc:    b9400be0     ldr    w0, [sp, #8]        //
    4005d0:    f9400bfe     ldr    x30, [sp, #16]      //
    4005d4:    910083ff     add    sp, sp, #0x20       //
    4005d8:    d65f03c0     ret                        //
```

<div style="page-break-after:always;"></div>

## Extra practice

Q3: _The following C code was compiled into assembly (using `gcc`). Label each line of assembly code with the line number of the line of C code from which the assembly instruction was derived._ 

In [ ]:
/*1*/   int abs(int value) {
/*2*/       if (value < 0) {
/*3*/           value = value * -1;
/*4*/       }
/*5*/       return value;
/*6*/   }

```
000000000000074c <abs>:
    74c:   d10043ff        sub     sp, sp, #0x10    // 1
    750:   b9000fe0        str     w0, [sp, #12]    // 1
    754:   b9400fe0        ldr     w0, [sp, #12]    // 2
    758:   7100001f        cmp     w0, #0x0         // 2 
    75c:   5400008a        b.ge    76c <abs+0x20>   // 2
    760:   b9400fe0        ldr     w0, [sp, #12]    // 3
    764:   4b0003e0        neg     w0, w0           // 3
    768:   b9000fe0        str     w0, [sp, #12]    // 3
    76c:   b9400fe0        ldr     w0, [sp, #12]    // 5
    770:   910043ff        add     sp, sp, #0x10    // 5
    774:   d65f03c0        ret                      // 5
``` 

```
000000000000074c <abs>:
    74c:   d10043ff        sub     sp, sp, #0x10    // 
    750:   b9000fe0        str     w0, [sp, #12]    // 
    754:   b9400fe0        ldr     w0, [sp, #12]    // 
    758:   7100001f        cmp     w0, #0x0         // 
    75c:   5400008a        b.ge    76c <abs+0x20>   // 
    760:   b9400fe0        ldr     w0, [sp, #12]    // 
    764:   4b0003e0        neg     w0, w0           // 
    768:   b9000fe0        str     w0, [sp, #12]    // 
    76c:   b9400fe0        ldr     w0, [sp, #12]    // 
    770:   910043ff        add     sp, sp, #0x10    // 
    774:   d65f03c0        ret                      // 
``` 

Q4: _The following C code was compiled into assembly (using `clang`). For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

In [ ]:
/* 1*/  int sum(int a, int b) {
/* 2*/      int c = a + b;
/* 3*/      return c;
/* 4*/  }
/* 5*/  int triple(int u, int v) {
/* 6*/      int r = 3;
/* 7*/      int s = sum(u, v);
/* 8*/      int t = s * r;
/* 9*/      return t;
/*10*/  }
/*11*/  int main() {
/*12*/      int x = 2;
/*13*/      int y = 3;
/*14*/      int z = triple(x, y);
/*15*/      return z;
/*16*/  }

```
0000000000400544 <sum>:
    400544:    d10043ff     sub    sp, sp, #0x10   // 1 
    400548:    b9000fe0     str    w0, [sp, #12]   // 1
    40054c:    b9000be1     str    w1, [sp, #8]    // 1
    400550:    b9400fe8     ldr    w8, [sp, #12]   // 2
    400554:    b9400be9     ldr    w9, [sp, #8]    // 2
    400558:    0b090108     add    w8, w8, w9      // 2
    40055c:    b90007e8     str    w8, [sp, #4]    // 2
    400560:    b94007e0     ldr    w0, [sp, #4]    // 3
    400564:    910043ff     add    sp, sp, #0x10   // 3
    400568:    d65f03c0     ret                    // 3
000000000040056c <triple>:
    40056c:    d100c3ff     sub    sp, sp, #0x30   // 5
    400570:    f90013fe     str    x30, [sp, #32]  // 5
    400574:    b9001fe0     str    w0, [sp, #28]   // 5
    400578:    b9001be1     str    w1, [sp, #24]   // 5
    40057c:    52800068     mov    w8, #0x3        // 6
    400580:    b90017e8     str    w8, [sp, #20]   // 6
    400584:    b9401fe0     ldr    w0, [sp, #28]   // 7
    400588:    b9401be1     ldr    w1, [sp, #24]   // 7
    40058c:    97ffffee     bl     400544 <sum>    // 7
    400590:    b90013e0     str    w0, [sp, #16]   // 7
    400594:    b94013e8     ldr    w8, [sp, #16]   // 8
    400598:    b94017e9     ldr    w9, [sp, #20]   // 8
    40059c:    1b097d08     mul    w8, w8, w9      // 8
    4005a0:    b9000fe8     str    w8, [sp, #12]   // 8
    4005a4:    b9400fe0     ldr    w0, [sp, #12]   // 9
    4005a8:    f94013fe     ldr    x30, [sp, #32]  // 9
    4005ac:    9100c3ff     add    sp, sp, #0x30   // 9
    4005b0:    d65f03c0     ret                    // 9
00000000004005b4 <main>:
    4005b4:    d10083ff     sub    sp, sp, #0x20   // 11
    4005b8:    f9000bfe     str    x30, [sp, #16]  // 11
    4005bc:    52800048     mov    w8, #0x2        // 12
    4005c0:    b9000be8     str    w8, [sp, #8]    // 12
    4005c4:    52800069     mov    w9, #0x3        // 13
    4005c8:    b90007e9     str    w9, [sp, #4]    // 13
    4005cc:    b9400be0     ldr    w0, [sp, #8]    // 14
    4005d0:    b94007e1     ldr    w1, [sp, #4]    // 14
    4005d4:    97ffffe5     bl    410056c <triple> // 14
    4005d8:    b90003e0     str    w0, [sp]        // 14
    4005dc:    b94003e0     ldr    w0, [sp]        // 15
    4005e0:    f9400bfe     ldr    x30, [sp, #16]  // 15
    4005e4:    910083ff     add    sp, sp, #0x20   // 15
    4005e8:    d65f03c0     ret                    // 15
```

```
0000000000400544 <sum>:
    400544:    d10043ff     sub    sp, sp, #0x10   // 
    400548:    b9000fe0     str    w0, [sp, #12]   // 
    40054c:    b9000be1     str    w1, [sp, #8]    // 
    400550:    b9400fe8     ldr    w8, [sp, #12]   // 
    400554:    b9400be9     ldr    w9, [sp, #8]    // 
    400558:    0b090108     add    w8, w8, w9      // 
    40055c:    b90007e8     str    w8, [sp, #4]    // 
    400560:    b94007e0     ldr    w0, [sp, #4]    // 
    400564:    910043ff     add    sp, sp, #0x10   // 
    400568:    d65f03c0     ret                    // 
000000000040056c <triple>:
    40056c:    d100c3ff     sub    sp, sp, #0x30   // 
    400570:    f90013fe     str    x30, [sp, #32]  // 
    400574:    b9001fe0     str    w0, [sp, #28]   // 
    400578:    b9001be1     str    w1, [sp, #24]   // 
    40057c:    52800068     mov    w8, #0x3        // 
    400580:    b90017e8     str    w8, [sp, #20]   // 
    400584:    b9401fe0     ldr    w0, [sp, #28]   // 
    400588:    b9401be1     ldr    w1, [sp, #24]   // 
    40058c:    97ffffee     bl     400544 <sum>    //  
    400590:    b90013e0     str    w0, [sp, #16]   // 
    400594:    b94013e8     ldr    w8, [sp, #16]   // 
    400598:    b94017e9     ldr    w9, [sp, #20]   // 
    40059c:    1b097d08     mul    w8, w8, w9      // 
    4005a0:    b9000fe8     str    w8, [sp, #12]   // 
    4005a4:    b9400fe0     ldr    w0, [sp, #12]   // 
    4005a8:    f94013fe     ldr    x30, [sp, #32]  // 
    4005ac:    9100c3ff     add    sp, sp, #0x30   // 
    4005b0:    d65f03c0     ret                    // 
00000000004005b4 <main>:
    4005b4:    d10083ff     sub    sp, sp, #0x20   // 
    4005b8:    f9000bfe     str    x30, [sp, #16]  // 
    4005bc:    52800048     mov    w8, #0x2        // 
    4005c0:    b9000be8     str    w8, [sp, #8]    // 
    4005c4:    52800069     mov    w9, #0x3        // 
    4005c8:    b90007e9     str    w9, [sp, #4]    // 
    4005cc:    b9400be0     ldr    w0, [sp, #8]    // 
    4005d0:    b94007e1     ldr    w1, [sp, #4]    // 
    4005d4:    97ffffe5     bl     40056c <triple> //  
    4005d8:    b90003e0     str    w0, [sp]        // 
    4005dc:    b94003e0     ldr    w0, [sp]        // 
    4005e0:    f9400bfe     ldr    x30, [sp, #16]  // 
    4005e4:    910083ff     add    sp, sp, #0x20   // 
    4005e8:    d65f03c0     ret                    // 
```

Q5: _Draw the contents of the stack and registers just prior to the execution of the second-to-last assembly instruction in `sum`. Assume the registers have the following initial values:_

* `pc` = `0x4005b4` 
* `sp` = `0xF80`
* `x30` = `0x400488`